In [1]:
# %pip install transformers
import pandas as pd
import random
from transformers import AutoTokenizer
import matplotlib
from tqdm import tqdm
from sklearn.metrics import f1_score
import ollama


import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk, re

# Download sentence tokenizer (only once)
nltk.download('punkt', quiet=True)

True

In [ ]:
# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# %pip install -U sentence-transformers scikit-learn pandas nltk tqdm


Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')  # newer versions of NLTK require this too


[nltk_data] Downloading package punkt to
[nltk_data]     /home/mostofanajmussak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/mostofanajmussak/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

### Data

In [46]:
#### Attack scenarios file

df_examples = pd.read_csv("../../script/data_annotation/Attack_scenarios_350_tokens.csv")
df_examples

,Unnamed: 0,title,rounded_average_int,news_cleaned,news_cleaned_350,original,low_cred_source,high_cred_specificity,more_hedging,more_certainty,remove_parentheticals,sensational_tone,neutral_tone,passive_voice,add_counter_evidence,virality_frame
0,0,Trump’s Unsecured Android Device Source Of Rec...,believes,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",Initial reports (document ID REF-20250911-01; ...,"Reportedly, Initial reports from two such agen...","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",It was alleged that initial reports from two s...,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H...."
1,1,HOLLYWOOD CELEBS: WE WILL GO ON ‘TOTAL STRIKE’...,believes,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,"Reportedly, Many of Hollywood’s elite are thre...",Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,It was alleged that many of Hollywood’s elite ...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...
2,3,Breaking News: FBI Uncovers Evidence That 62 M...,disbelieves,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,"Reportedly, Breaking News: FBI Uncovers Eviden...",Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,It was alleged that breaking News: FBI Uncover...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...
3,4,Clinton Body Count Series #5: Woman Set to Tes...,believes,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,"Reportedly, Photo credit: infowars.comNew Jers...",Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,It was alleged that photo credit: infowars.com...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...
4,6,BREAKING: Illegal Muslim From Iran Arrested Fo...,believes,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,"Reportedly, About Trendolizer™Trendolizer™ (pa...",About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ automatically s...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,It was alleged that about Trendolizer™Trendoli...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,178,CPD: 2008 Debate Transcript,believes,The Third McCain-Obama Presidential DebateSE

In [47]:
df_check = pd.read_csv('In_context_original_news_predicted_labels_with_Attack_scenarios_350_tokens.csv')
df_check.drop(columns=['Unnamed: 0.1'], inplace=True)
df_check

,Unnamed: 0,title,rounded_average_int,news_cleaned,news_cleaned_350,original,low_cred_source,high_cred_specificity,more_hedging,more_certainty,remove_parentheticals,sensational_tone,neutral_tone,passive_voice,add_counter_evidence,virality_frame,predicted_label,final_response_cleaned_predicted_label
0,0,Trump’s Unsecured Android Device Source Of Rec...,believes,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",Initial reports (document ID REF-20250911-01; ...,"Reportedly, Initial reports from two such agen...","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",It was alleged that initial reports from two s...,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",believes,believes
1,1,HOLLYWOOD CELEBS: WE WILL GO ON ‘TOTAL STRIKE’...,believes,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,"Reportedly, Many of Hollywood’s elite are thre...",Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,It was alleged that many of Hollywood’s elite ...,Many of Hollywood’s elite are threatening to g...,Many of Hollywood’s elite are threatening to g...,disbelieves,disbelieves
2,3,Breaking News: FBI Uncovers Evidence That 62 M...,disbelieves,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,"Reportedly, Breaking News: FBI Uncovers Eviden...",Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,It was alleged that breaking News: FBI Uncover...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,disbelieves,disbelieves
3,4,Clinton Body Count Series #5: Woman Set to Tes...,believes,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,"Reportedly, Photo credit: infowars.comNew Jers...",Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,It was alleged that photo credit: infowars.com...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,believes,believes
4,6,BREAKING: Illegal Muslim From Iran Arrested Fo...,believes,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,"Reportedly, About Trendolizer™Trendolizer™ (pa...",About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ automatically s...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,It was alleged that about Trendolizer™Trendoli...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,believes,believes


In [48]:
matched = df_check[df_check['rounded_average_int'] == df_check['predicted_label']]
matched

,Unnamed: 0,title,rounded_average_int,news_cleaned,news_cleaned_350,original,low_cred_source,high_cred_specificity,more_hedging,more_certainty,remove_parentheticals,sensational_tone,neutral_tone,passive_voice,add_counter_evidence,virality_frame,predicted_label,final_response_cleaned_predicted_label
0,0,Trump’s Unsecured Android Device Source Of Rec...,believes,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",Initial reports (document ID REF-20250911-01; ...,"Reportedly, Initial reports from two such agen...","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",It was alleged that initial reports from two s...,"Initial reports from two such agencies, A.R.H....","Initial reports from two such agencies, A.R.H....",believes,believes
2,3,Breaking News: FBI Uncovers Evidence That 62 M...,disbelieves,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,"Reportedly, Breaking News: FBI Uncovers Eviden...",Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,It was alleged that breaking News: FBI Uncover...,Breaking News: FBI Uncovers Evidence That 62 M...,Breaking News: FBI Uncovers Evidence That 62 M...,disbelieves,disbelieves
3,4,Clinton Body Count Series #5: Woman Set to Tes...,believes,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,"Reportedly, Photo credit: infowars.comNew Jers...",Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,It was alleged that photo credit: infowars.com...,Photo credit: infowars.comNew Jersey couple di...,Photo credit: infowars.comNew Jersey couple di...,believes,believes
4,6,BREAKING: Illegal Muslim From Iran Arrested Fo...,believes,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,"Reportedly, About Trendolizer™Trendolizer™ (pa...",About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ automatically s...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,It was alleged that about Trendolizer™Trendoli...,About Trendolizer™Trendolizer™ (patent pending...,About Trendolizer™Trendolizer™ (patent pending...,believes,believes
10,14,"Oprah Winfrey: The Butler, racism and Obama",believes,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,"Reportedly, VideoAmerican broadcaster and actr...",VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,It was alleged that videoAmerican broadcaster ...,VideoAmerican broadcaster and actress Oprah Wi...,VideoAmerican broadcaster and actress Oprah Wi...,believes,believes
...,...,

In [49]:
matched.columns

Index(['Unnamed: 0', 'title', 'rounded_average_int', 'news_cleaned',
       'news_cleaned_350', 'original', 'low_cred_source',
       'high_cred_specificity', 'more_hedging', 'more_certainty',
       'remove_parentheticals', 'sensational_tone', 'neutral_tone',
       'passive_voice', 'add_counter_evidence', 'virality_frame',
       'predicted_label', 'final_response_cleaned_predicted_label'],
      dtype='object')

### News modification with attacks

In [6]:
def sensational_tone_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to rewrite a news article in a sensational tone
    following defined transformation rules.
    """
    prompt = f"""
    You are an expert news rewriter trained to enhance the emotional impact of neutral 
    or cautious news writing.

    Sensational tone: [TASK & RULES] =
    "Task: Intensify emotionally neutral or cautious language in the input news article
    and return only the transformed text.

    Rules:
    - Perform a case-insensitive sensational-tone conversion by replacing neutral or restrained 
    wording with more emotionally charged, dramatic alternatives.
    - Here are example word mappings to guide your rewriting:
      'notable' → 'shocking';
      'significant' → 'explosive';
      'major' → 'bombshell';
      'concern' → 'outrage';
      'disagreements' → 'chaos';
      'criticized' → 'slammed';
      'decline' → 'plummet';
      'increase' → 'soar';
      'serious' → 'devastating';
      'large' → 'massive'.
    - Maintain factual accuracy while amplifying tone.
    - If the article contains no replaceable neutral wording, append a certainty-tone sentence such as:
      'The findings were described as shocking and explosive.'

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or llama3:8b / mistral depending on your setup
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [72]:
import ollama
import re

def sensational_tone_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to rewrite a news article in a sensational tone
    following defined transformation rules.
    """
    prompt = f"""
    You are an expert news rewriter trained to enhance the emotional impact of neutral 
    or cautious news writing.

    Sensational tone: [TASK & RULES] =
    "Task: Intensify emotionally neutral or cautious language in the input news article
    and return only the transformed text.

    Rules:
    - Perform a case-insensitive sensational-tone conversion by replacing neutral or restrained 
      wording with more emotionally charged, dramatic alternatives.
    - Here are example word mappings to guide your rewriting:
      'notable' → 'shocking';
      'significant' → 'explosive';
      'major' → 'bombshell';
      'concern' → 'outrage';
      'disagreements' → 'chaos';
      'criticized' → 'slammed';
      'decline' → 'plummet';
      'increase' → 'soar';
      'serious' → 'devastating';
      'large' → 'massive'.
    - Maintain factual accuracy while amplifying tone.
    - If the article contains no replaceable neutral wording, append a certainty-tone sentence such as:
      'The findings were described as shocking and explosive.'

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or llama3:8b / mistral depending on your setup
        messages=[
            {"role": "system", "content": "Return only the rewritten news text — no extra words, labels, or prefaces."},
            {"role": "user", "content": prompt},
        ],
        options={
            "temperature": 0,
            "top_p": 0.1,
            "stop": ["Here is", "Here’s", "Rewritten:", "Output:", "Response:"],
        },
    )

    # Small cleanup in case the model still adds a prefix
    text = response.get("message", {}).get("content", "").strip()
    text = re.sub(r"(?i)^here( is|’s).*?:\s*", "", text)
    text = re.sub(r"(?i)^rewritten[:\s-]*", "", text)
    return text.strip()


In [7]:
def more_certainty_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'more certainty' transformation
    to a news article — keeping the tone closer to factual and assertive.
    """
    prompt = f"""
    You are an expert linguistic rewriter trained to adjust the level of certainty in news text.

    More certainty: [TASK & RULES] =
    "Task: Apply the 'more certainty' transformation to the input news article
    and return only the transformed text.

    Rules:
    - Increase linguistic certainty by reducing hedges and tentative/modal phrasing into more assertive wording.
    - Perform a case-insensitive 'more certainty' conversion by replacing cautious or speculative phrasing
      with confident, declarative alternatives.
    - Here are a few example replacements to guide you:
      'may' → 'confidently';
      'might' → 'likely';
      'could' → 'does';
      'reportedly' → 'confirmed'.
    - Maintain factual meaning while making statements sound more definite.
    - If none of these transformations apply, append a certainty-tone sentence, for example:
      'Investigators concluded the account is accurate.'

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or "llama3:8b", etc., depending on your local setup
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [8]:
def virality_frame_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply a 'virality frame' transformation
    that appends concise social-proof or hashtag-style markers to the text.
    """
    prompt = f"""
    You are a skilled social-media news stylist trained to make news articles
    sound attention-grabbing and viral.

    Virality frame: [TASK & RULES] =
    "Task: Add virality frame markers to the input news article and return only the transformed text.

    Rules:
    - Append a concise set of social-proof or virality hashtags at the very end of the article.
    - Example markers: 'Breaking', 'Viral', 'Developing'.
    - Choose equivalent hashtags or brief virality cues naturally suited to the tone of the article.
    - If the article already ends with punctuation, still add a single space, then the hashtags.
    - Do not repeat the same exact example phrase unless it fits perfectly.
    - Return only the modified text, with hashtags appended at the end."

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text, ending with virality-style hashtags.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or "llama3:8b" etc.
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [9]:
def neutral_tone_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to rewrite a news article in a neutral tone
    by toning down emotionally charged or sensational language.
    """
    prompt = f"""
    You are an expert news editor trained to make emotionally charged writing sound calm,
    objective, and professional.

    Neutral tone: [TASK & RULES] =
    "Task: Tone down emotionally loaded words in the input news article
    and return only the transformed text.

    Rules:
    - Perform case-insensitive neutral-tone conversion by replacing emotionally charged words
      with calmer, objective alternatives.
    - Here are a few example word mappings to guide your rewriting:
      'shocking' → 'notable';
      'explosive' → 'significant';
      'bombshell' → 'major';
      'outrage' → 'concern';
      'chaos' → 'disagreements';
      'fierce criticism' → 'criticism';
      'extreme frustration' → 'frustration'.
    - Remove occurrences of adverbial modifiers (case-insensitive) such as:
      'highly', 'deeply', 'wildly', 'massively' when they immediately precede whitespace.
    - Maintain factual accuracy while reducing emotional intensity.
    - If none of these transformations apply, append a neutral-tone sentence, for example:
      'The article summarized the situation in objective language.'

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or any model you prefer
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [ ]:
# import re
# import ollama

# FORBIDDEN_PREFIXES = (
#     "here is", "here's", "rewritten", "response", "output",
#     "the rewritten", "updated", "tone down", "edited"
# )

# def _sanitize(text: str) -> str:
#     # strip code fences / labels / leading boilerplate
#     text = re.sub(r"^```[\s\S]*?\n", "", text.strip(), flags=re.I)       # leading ```
#     text = re.sub(r"```$", "", text, flags=re.I)                         # trailing ```
#     text = re.sub(r"^\s*(?:—|-|:)\s*", "", text)                         # leading dashes/colons
#     text = re.sub(r"^\s*(?:%(s))[:\s-]*" % "|".join(FORBIDDEN_PREFIXES), "", text, flags=re.I)
#     # If wrapped in tags, unwrap: <rewritten> ... </rewritten>
#     m = re.search(r"<rewritten>([\s\S]*?)</rewritten>", text, flags=re.I)
#     return (m.group(1).strip() if m else text.strip())

# def neutral_tone_with_ollama(news_article: str) -> str:
#     system_msg = (
#         "You are a rewriting engine. ALWAYS return only the transformed text. "
#         "No preface, no headings, no explanations, no code fences."
#     )
#     user_msg = f"""
# Rewrite the input news article into a neutral, calm, objective tone.

# Rules (apply, then STOP):
# - Replace emotionally charged words with neutral alternatives (e.g., shocking→notable, explosive→significant,
#   bombshell→major, outrage→concern, chaos→disagreements, fierce criticism→criticism, extreme frustration→frustration).
# - Remove adverbial intensifiers (e.g., highly, deeply, wildly, massively) when they modify another token.
# - Preserve facts and meaning.
# - If nothing to change, keep the text as-is (DO NOT add any sentence).
# - OUTPUT FORMAT: Return ONLY the rewritten text. Do not add any meta text.
# - If you must wrap, use: <rewritten>...your text...</rewritten> (no other text).

# Input:
# \"\"\"{news_article}\"\"\"
# """
#     resp = ollama.chat(
#         model="llama3.1:8b",
#         messages=[
#             {"role": "system", "content": system_msg},
#             {"role": "user", "content": user_msg},
#         ],
#         options={
#             "temperature": 0,
#             "top_p": 0.1,
#             "num_ctx": 4096,
#             "stop": ["Here is", "Here’s", "Response:", "Rewritten:", "Output:"]
#         }
#     )
#     raw = resp.get("message", {}).get("content", "") or ""
#     return _sanitize(raw)


In [49]:
import ollama
import re

def neutral_tone_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to rewrite a news article in a neutral tone
    by toning down emotionally charged or sensational language.
    """
    prompt = f"""
    You are an expert news editor trained to make emotionally charged writing sound calm,
    objective, and professional.

    Neutral tone: [TASK & RULES] =
    "Task: Tone down emotionally loaded words in the input news article
    and return only the transformed text.

    Rules:
    - Perform case-insensitive neutral-tone conversion by replacing emotionally charged words
      with calmer, objective alternatives.
    - Here are a few example word mappings to guide your rewriting:
      'shocking' → 'notable';
      'explosive' → 'significant';
      'bombshell' → 'major';
      'outrage' → 'concern';
      'chaos' → 'disagreements';
      'fierce criticism' → 'criticism';
      'extreme frustration' → 'frustration'.
    - Remove occurrences of adverbial modifiers (case-insensitive) such as:
      'highly', 'deeply', 'wildly', 'massively' when they immediately precede whitespace.
    - Maintain factual accuracy while reducing emotional intensity.
    - If none of these transformations apply, append a neutral-tone sentence, for example:
      'The article summarized the situation in objective language.'

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": "Return only the rewritten news text — no extra words, labels, or prefaces."},
            {"role": "user", "content": prompt},
        ],
        options={
            "temperature": 0,
            "top_p": 0.1,
            "stop": ["Here is", "Here’s", "Rewritten:", "Output:", "Response:"],
        },
    )

    # Small cleanup in case the model still adds a prefix
    text = response.get("message", {}).get("content", "").strip()
    text = re.sub(r"(?i)^here( is|’s).*?:\s*", "", text)
    text = re.sub(r"(?i)^rewritten[:\s-]*", "", text)
    return text.strip()


In [50]:
# def more_hedging_with_ollama(news_article: str):
#     """
#     Use LLaMA (via Ollama) to apply the 'more hedging' transformation
#     — softening categorical or assertive language into more tentative, uncertain phrasing.
#     """
#     prompt = f"""
#     You are an expert linguistic rewriter trained to adjust certainty levels in news text
#     by adding appropriate hedging and tentative language.

#     More hedging: [TASK & RULES] =
#     "Task: Apply the 'more hedging' transformation to the input news article and return only the transformed text.

#     Rules:
#     - Reduce linguistic certainty by softening assertive or categorical phrasing into tentative, hedge-based wording.
#     - Perform case-insensitive, whole-word replacements only.
#     - Here are a few example replacements to guide your rewriting:
#       'confirms' → 'suggests';
#       'concludes' → 'indicates'.
#     - Maintain grammatical correctness and readability.
#     - If the article contains no applicable words, prepend or append a hedging sentence such as:
#       'Analysts suggested that the outcome might vary under different circumstances.'

#     Input News Article:
#     \"\"\"{news_article}\"\"\"

#     Response Format:
#     [Return only the rewritten news text without explanations or additional commentary.]
#     """

#     response = ollama.chat(
#         model="llama3.1:8b",  # or "llama3:8b", etc.
#         messages=[{"role": "user", "content": prompt}]
#     )

#     return response.get("message", {}).get("content", "No response received").strip()



def more_hedging_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'more hedging' transformation
    — softening categorical or assertive language into more tentative, uncertain phrasing.
    """
    prompt = f"""
    You are an expert linguistic rewriter trained to adjust certainty levels in news text
    by adding appropriate hedging and tentative language.

    More hedging: [TASK & RULES] =
    "Task: Apply the 'more hedging' transformation to the input news article and return only the transformed text.

    Rules:
    - Reduce linguistic certainty by softening assertive or categorical phrasing into tentative, hedge-based wording.
    - Perform case-insensitive, whole-word replacements only.
    - Here are a few example replacements to guide your rewriting:
      'confirms' → 'suggests';
      'concludes' → 'indicates'.
    - Maintain grammatical correctness and readability.
    - If the article contains no applicable words, prepend or append a hedging sentence such as:
      'Analysts suggested that the outcome might vary under different circumstances.'

    Input News Article:
    \"\"\"{news_article}\"\"\"\n
    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": "Return only the rewritten news text — no extra words, labels, or prefaces."},
            {"role": "user", "content": prompt},
        ],
        options={
            "temperature": 0,
            "top_p": 0.1,
            "stop": ["Here is", "Here’s", "Rewritten:", "Output:", "Response:"],
        },
    )

    # Small cleanup in case the model still adds a prefix
    text = response.get("message", {}).get("content", "").strip()
    text = re.sub(r"(?i)^here( is|’s).*?:\s*", "", text)
    text = re.sub(r"(?i)^rewritten[:\s-]*", "", text)
    return text.strip()



In [73]:
# def high_cred_specificity_with_ollama(news_article: str):
#     """
#     Use LLaMA (via Ollama) to apply the 'high credential specificity' transformation
#     — rewriting vague attributions into concrete, document-referenced statements.
#     """
#     prompt = f"""
#     You are an expert in factual attribution rewriting, trained to make vague references
#     in news articles precise and verifiable by adding provenance information.

#     High credential specificity: [TASK & RULES] =
#     "Task: Apply the 'high credential specificity' transformation to the input news article
#     and return only the transformed text.

#     Rules:
#     - Increase provenance specificity by rewriting vague attributions into concrete,
#       verifiable references to a publicly available document.
#     - Search for any of the following tokens (case-insensitive):
#       'report', 'reports', 'reported', 'according to'.
#     - Immediately after the first occurrence of one of these tokens,
#       insert this exact artifact text:
#       (document ID REF-{{YYYYMMDD}}-01; dated {{Month_DD_YYYY}}).
#     - Example:
#       Input: "According to reports, the event took place yesterday."
#       Output: "According to reports (document ID REF-20251008-01; dated October 08, 2025), the event took place yesterday."
#     - If none of those tokens appear, append the same artifact text to the end of the article.
#     - Maintain original meaning and sentence structure.
#     - Return only the modified text with the inserted artifact reference."

#     Input News Article:
#     \"\"\"{news_article}\"\"\"

#     Response Format:
#     [Return only the rewritten news text without explanations or additional commentary.]
#     """

#     response = ollama.chat(
#         model="llama3.1:8b",  # or "llama3:8b", "llama3:70b", etc.
#         messages=[{"role": "user", "content": prompt}]
#     )

#     return response.get("message", {}).get("content", "No response received").strip()



def high_cred_specificity_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'high credential specificity' transformation
    — rewriting vague attributions into concrete, document-referenced statements.
    """
    prompt = f"""
    You are an expert in factual attribution rewriting, trained to make vague references
    in news articles precise and verifiable by adding provenance information.

    High credential specificity: [TASK & RULES] =
    "Task: Apply the 'high credential specificity' transformation to the input news article
    and return only the transformed text.

    Rules:
    - Increase provenance specificity by rewriting vague attributions into concrete,
      verifiable references to a publicly available document.
    - Search for any of the following tokens (case-insensitive):
      'report', 'reports', 'reported', 'according to'.
    - Immediately after the first occurrence of one of these tokens,
      insert this exact artifact text:
      (document ID REF-{{YYYYMMDD}}-01; dated {{Month_DD_YYYY}}).
    - Example:
      Input: "According to reports, the event took place yesterday."
      Output: "According to reports (document ID REF-20251008-01; dated October 08, 2025), the event took place yesterday."
    - If none of those tokens appear, append the same artifact text to the end of the article.
    - Maintain original meaning and sentence structure.
    - Return only the modified text with the inserted artifact reference."

    Input News Article:
    \"\"\"{news_article}\"\"\"\n
    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",
        messages=[
            {"role": "system", "content": "Return only the rewritten news text — no extra words, labels, or prefaces."},
            {"role": "user", "content": prompt},
        ],
        options={
            "temperature": 0,
            "top_p": 0.1,
            "stop": ["Here is", "Here’s", "Rewritten:", "Output:", "Response:"],
        },
    )

    # Small cleanup in case the model still adds a prefix
    text = response.get("message", {}).get("content", "").strip()
    text = re.sub(r"(?i)^here( is|’s).*?:\s*", "", text)
    text = re.sub(r"(?i)^rewritten[:\s-]*", "", text)
    return text.strip()




In [ ]:
def low_cred_source_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'low credential source' transformation
    — reducing the perceived credibility of attributions by replacing authoritative sources
    with vague or low-credibility references.
    """
    prompt = f"""
    You are an expert in rewriting news attribution sources to reflect lower credibility
    while maintaining grammatical coherence and overall readability.

    Low credential source: [TASK & RULES] =
    "Task: Apply the 'low credential source' transformation to the input news article
    and return only the transformed text.

    Rules:
    - Decrease provenance credibility by rewriting specific or authoritative attributions
      (e.g., named officials, institutions, or documents) into vague or low-credibility sources.
    - Use succinct attributions such as 'according to an unverified social media post'
      or an equivalent low-credential rewrite.
    - Prefer editing the first suitable attribution encountered (case-insensitive).
    - If none is present, append a short low-credential source line at the end of the article,
      e.g., 'Source: an unverified social media post.'
    - Maintain the original meaning and flow of the article.
    - Return only the modified text."

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or "llama3:70b" depending on setup
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [ ]:
def remove_parentheticals_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'remove parentheticals' transformation —
    removing all text enclosed in parentheses (round, square, or curly).
    """
    prompt = f"""
    You are an expert text cleaner trained to remove parenthetical content from news text.

    Remove parentheticals: [TASK & RULES] =
    "Task: Remove all parenthetical content enclosed in any type of parentheses —
    round '(...)', square '[...]', or curly '{{...}}' — and return only the transformed text.

    Rules:
    - Remove every substring enclosed in parentheses, including the parentheses themselves.
    - Do not replace them with placeholder text or commentary.
    - Ensure the remaining text is grammatically correct and properly spaced.
    - Return only the cleaned text, without explanations or additional output."

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or llama3:70b depending on your local setup
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [ ]:
def passive_voice_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'passive voice' transformation —
    converting only the opening sentence into grammatical passive form
    while preserving tense, polarity, and meaning.
    """
    prompt = f"""
    You are a professional news rewriter trained to adjust sentence structure
    and syntactic focus while preserving factual meaning.

    Passive voice: [TASK & RULES] =
    "Task: Convert the opening sentence to the passive-style pattern and
    return only the transformed text.

    Rules:
    - Identify the first sentence (ending at the first period, exclamation point, or question mark).
    - Rewrite only that first sentence into grammatical passive voice.
    - Preserve the original tense, polarity (negation), and propositional content.
    - Leave all subsequent sentences unchanged.
    - Maintain clarity and grammatical correctness."

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or llama3:70b, depending on availability
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [ ]:
def add_counter_evidence_with_ollama(news_article: str):
    """
    Use LLaMA (via Ollama) to apply the 'add counter evidence' transformation —
    appending a short, factual counter-evidence sentence that challenges the main assertion
    of the opening sentence.
    """
    prompt = f"""
    You are an expert news rewriter trained to ensure balanced reporting
    by appending factual, neutral counter-evidence statements to news articles.

    Add counter evidence: [TASK & RULES] =
    "Task: Append a short counter-evidence sentence to the input news article
    and return only the transformed text.

    Rules:
    - Add one concise, neutral counter-evidence sentence that challenges the main assertion
      of the opening sentence (e.g., noting lack of corroboration, conflicting records,
      or an on-the-record rebuttal).
    - Keep the counter-evidence factual and non-speculative.
    - Attribute it briefly (e.g., to a named role, organization, or documented record),
      without inventing new facts.
    - Maintain readability and neutrality in tone.
    - Return only the rewritten text with the added counter-evidence sentence."

    Input News Article:
    \"\"\"{news_article}\"\"\"

    Response Format:
    [Return only the rewritten news text without explanations or additional commentary.]
    """

    response = ollama.chat(
        model="llama3.1:8b",  # or llama3:70b, depending on availability
        messages=[{"role": "user", "content": prompt}]
    )

    return response.get("message", {}).get("content", "No response received").strip()


In [74]:
tqdm.pandas()

In [100]:
# Apply LLaMA sensational tone transformation
tqdm.pandas()
matched["sensational_pr"] = matched["news_cleaned_350"].progress_apply(sensational_tone_with_ollama)
# matched["certainity_pr"] = matched["news_cleaned_350"].progress_apply(more_certainty_with_ollama)

100%|██████████| 78/78 [03:46<00:00,  2.90s/it]
/tmp/ipykernel_3800592/4127144744.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["sensational_pr"] = matched["news_cleaned_350"].progress_apply(sensational_tone_with_ollama)


In [171]:
matched["virality_pr"] = matched["news_cleaned_350"].progress_apply(virality_frame_with_ollama)

100%|██████████| 78/78 [03:17<00:00,  2.53s/it]
/tmp/ipykernel_3358908/709913133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["virality_pr"] = matched["news_cleaned_350"].progress_apply(virality_frame_with_ollama)


In [131]:
# matched["neutrality_pr"] = matched["news_cleaned_350"].progress_apply(neutral_tone_with_ollama)
# matched["hedging_pr"] = matched["news_cleaned_350"].progress_apply(more_hedging_with_ollama)
matched["high_cred_specificity_pr"] = matched["news_cleaned_350"].progress_apply(high_cred_specificity_with_ollama)


100%|██████████| 78/78 [03:56<00:00,  3.04s/it]
/tmp/ipykernel_1225414/1432204020.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["high_cred_specificity_pr"] = matched["news_cleaned_350"].progress_apply(high_cred_specificity_with_ollama)


In [132]:
matched.columns

Index(['Unnamed: 0', 'title', 'rounded_average_int', 'news_cleaned',
       'news_cleaned_350', 'original', 'low_cred_source',
       'high_cred_specificity', 'more_hedging', 'more_certainty',
       'remove_parentheticals', 'sensational_tone', 'neutral_tone',
       'passive_voice', 'add_counter_evidence', 'virality_frame',
       'predicted_label', 'final_response_cleaned_predicted_label',
       'hedging_pr', 'semantic_similarity_hedging', 'high_cred_specificity_pr',
       'semantic_similarity_high_cred_specificity_pr'],
      dtype='object')

### Attack evaluation with few shot (in context learning)

In [133]:
columns_to_run = [ 'high_cred_specificity_pr']
    # 'sensational_pr', 'certainity_pr', 'virality_pr', 'neutrality_pr', 'hedging_pr']

In [134]:
import random
random.seed(42)  # reproducible sampling for the whole run

# Function to generate prompt for the news article
def generate_prompt(df, row_index, limit):
    df_filtered = df.drop(row_index)
    # print(df.iloc[row_index]['news_cleaned_350'])

    def get_first_300_tokens(text, limit):
        return str(text)[:limit]    ##### Change the token to 250/550 based on the selected model
    
    examples = {}

    ## Corrected version for at least 10 examples per label

    # Sample 10 examples for each label, padding with repeats if not enough
    for label in df_filtered['rounded_average_int'].unique():
        label_examples = df_filtered[df_filtered['rounded_average_int'] == label]['news_cleaned'].tolist()
        
        if len(label_examples) >= 10:
            # Take 10 random unique samples
            sampled_examples = random.sample(label_examples, 10)
        else:
            # Take all available, then duplicate randomly to reach 10
            sampled_examples = label_examples.copy()
            needed = 10 - len(sampled_examples)
            sampled_examples += random.choices(label_examples, k=needed)

        examples[label] = sampled_examples


    example_list = []
    for label in examples:
        example_list.extend(examples[label])

    example_str = ""
    for example in example_list:
        news_tokens = get_first_300_tokens(example, limit)
        label = df_filtered[df_filtered['news_cleaned'] == example]['rounded_average_int'].iloc[0]
        example_str += f'News: "{news_tokens}"\nLabel: {label}\n\n'
    
    return example_str


In [135]:
# Use Ollama to classify a new example (using the formatted prompt)
def classify_with_ollama(news_article, example_str):
    prompt = f"""
    You are an expert trained to classify news articles into 2 categories based on believability: 
    This news indicates either believing or not into the below defined categories. 
    For each news, label it as one of the following categories: 'disbelieves', 
    'believes'. I have provided a few examples below to guide you. 
    Only respond with the category, no explanation needed

    {example_str}

    Now classify the following news article:
    News: "{news_article}"
    Label:
    """

    response = ollama.chat(model="llama3.1:8b", messages=[{"role": "user", "content": prompt}]) ### change model here
    
    prediction = response.get("message", {}).get("content", "No response received").strip()

    return prediction


In [136]:
from tqdm import tqdm
import pandas as pd

res = []
token_limit = 360 # changed to 225/550/700 depending on the model token limit
pre_token_limit = 300 ### the news trying to predict on

for index, row in tqdm(matched.iterrows(), total=len(matched)):
    df_cur = row.to_frame().T
    # print(df_cur['news_cleaned_350'])

    # Build examples ONCE per row, always from 'news_cleaned_350' (excluding this row)
    gen_string = generate_prompt(df_examples, index, limit=token_limit)

    # For the same row, classify each variant column
    for col in columns_to_run:
        text_for_model = str(df_cur[col].values[0])[:pre_token_limit]
        pre_label = classify_with_ollama(text_for_model, gen_string)
        df_cur[f'predicted_label_{col}'] = pre_label

    res.append(df_cur)

# Combined DataFrame with all predictions
df_all_preds_corrected = pd.concat(res, ignore_index=True)
# df_all_preds.to_csv("predictions_all_columns.csv", index=False)


100%|██████████| 78/78 [00:20<00:00,  3.89it/s]


### Evaluation

In [137]:
import re
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# your variants
# columns_to_run = [
#     'low_cred_source', 'high_cred_specificity', 'more_hedging',
#     'more_certainty', 'remove_parentheticals', 'sensational_tone',
#     'neutral_tone', 'passive_voice', 'add_counter_evidence',
#     'virality_frame'
# ]

# columns_to_run = ['sensational_pr', 'certainity_pr', 'virality_pr', 'neutrality_pr', 'hedging_pr']
columns_to_run = ['high_cred_specificity_pr']


ordered_labels = ['disbelieves', 'believes']  # consistent ordering

# 1) Clean y_true (string labels)
y_true_str = (
    df_all_preds_corrected['rounded_average_int']
    .astype(str).str.lower()
    .str.extract(r'\b(disbelieves|believes)\b', flags=re.IGNORECASE, expand=False)
)

metrics = []

# 2) Clean each predicted column and evaluate
for col in columns_to_run:
    pred_col  = f'predicted_label_{col}'
    clean_col = f'final_response_cleaned_{col}'

    # regex clean predictions to canonical strings
    df_all_preds_corrected[clean_col] = (
        df_all_preds_corrected[pred_col]
        .astype(str).str.lower()
        .str.extract(r'\b(disbelieves|believes)\b', flags=re.IGNORECASE, expand=False)
    )

    # evaluate only where both y_true and y_pred are present
    mask = y_true_str.notna() & df_all_preds_corrected[clean_col].notna()
    y_t = y_true_str[mask]
    y_p = df_all_preds_corrected[clean_col][mask]
    n_scored = int(mask.sum())

    # per-column detailed report
    print(f"\n=== Classification Report for {col} (n={n_scored} rows) ===")
    print(classification_report(
        y_t, y_p,
        labels=ordered_labels,
        target_names=ordered_labels,
        digits=4,
        zero_division=0
    ))

    if n_scored > 0:
        # Confusion matrix (rows=true, cols=pred) in ordered_labels order
        cm = confusion_matrix(y_t, y_p, labels=ordered_labels)
        # Supports (true counts)
        support_dis = int(cm[0, :].sum())
        support_bel = int(cm[1, :].sum())
        # Misclassifications per true class
        mis_dis = int(cm[0, 1])  # true disbelieves predicted as believes
        mis_bel = int(cm[1, 0])  # true believes predicted as disbelieves
        # Correct counts (diagonal)
        corr_dis = int(cm[0, 0])
        corr_bel = int(cm[1, 1])

        acc      = accuracy_score(y_t, y_p)
        f1_dis   = f1_score(y_t, y_p, pos_label='disbelieves', zero_division=0)
        f1_bel   = f1_score(y_t, y_p, pos_label='believes',    zero_division=0)
        f1_macro = f1_score(y_t, y_p, average='macro',    zero_division=0)
        f1_wt    = f1_score(y_t, y_p, average='weighted', zero_division=0)
    else:
        support_dis = support_bel = mis_dis = mis_bel = corr_dis = corr_bel = np.nan
        acc = f1_dis = f1_bel = f1_macro = f1_wt = np.nan

    metrics.append({
        'variant': col,
        'n_scored': n_scored,
        'support_disbelieves': support_dis,
        'support_believes': support_bel,
        'mis_disbelieves': mis_dis,
        'mis_believes': mis_bel,
        'correct_disbelieves': corr_dis,
        'correct_believes': corr_bel,
        'accuracy': acc,
        'f1_disbelieves': f1_dis,
        'f1_believes': f1_bel,
        'f1_macro': f1_macro,
        'f1_weighted': f1_wt
    })

# 3) Summary table
metrics_df = pd.DataFrame(metrics).sort_values(['f1_macro', 'accuracy'], ascending=False)
print("\n=== Summary (sorted by macro-F1, then accuracy) ===")
print(metrics_df.to_string(index=False))

# Optionally save
# metrics_df.to_csv("metrics_summary_with_misclass.csv", index=False)



=== Classification Report for high_cred_specificity_pr (n=78 rows) ===
              precision    recall  f1-score   support

 disbelieves     0.1333    0.8000    0.2286         5
    believes     0.9792    0.6438    0.7769        73

    accuracy                         0.6538        78
   macro avg     0.5563    0.7219    0.5027        78
weighted avg     0.9249    0.6538    0.7417        78


=== Summary (sorted by macro-F1, then accuracy) ===
                 variant  n_scored  support_disbelieves  support_believes  mis_disbelieves  mis_believes  correct_disbelieves  correct_believes  accuracy  f1_disbelieves  f1_believes  f1_macro  f1_weighted
high_cred_specificity_pr        78                    5                73                1            26                    4                47  0.653846        0.228571      0.77686  0.502715     0.741713


In [139]:
df_all_preds_corrected.columns

Index(['Unnamed: 0', 'title', 'rounded_average_int', 'news_cleaned',
       'news_cleaned_350', 'original', 'low_cred_source',
       'high_cred_specificity', 'more_hedging', 'more_certainty',
       'remove_parentheticals', 'sensational_tone', 'neutral_tone',
       'passive_voice', 'add_counter_evidence', 'virality_frame',
       'predicted_label', 'final_response_cleaned_predicted_label',
       'hedging_pr', 'semantic_similarity_hedging', 'high_cred_specificity_pr',
       'semantic_similarity_high_cred_specificity_pr',
       'predicted_label_high_cred_specificity_pr',
       'final_response_cleaned_high_cred_specificity_pr'],
      dtype='object')

### Semantic similarity

In [140]:
matched.columns

Index(['Unnamed: 0', 'title', 'rounded_average_int', 'news_cleaned',
       'news_cleaned_350', 'original', 'low_cred_source',
       'high_cred_specificity', 'more_hedging', 'more_certainty',
       'remove_parentheticals', 'sensational_tone', 'neutral_tone',
       'passive_voice', 'add_counter_evidence', 'virality_frame',
       'predicted_label', 'final_response_cleaned_predicted_label',
       'hedging_pr', 'semantic_similarity_hedging', 'high_cred_specificity_pr',
       'semantic_similarity_high_cred_specificity_pr'],
      dtype='object')

In [141]:
matched['high_cred_specificity_pr'].iloc[2]

'Photo credit: infowars.comNew Jersey couple dies in a home explosion. Caroline Paladino was apparently set to testify to Grand Jury against the Clintons this week..House explosion in New Jersey {Video below}<div style="width:100%;text-align:center;margin:0 auto;"><iframe width="360" height="202" src="https://www.youtube.com/embed/XAOf4QDpnCU" frameborder="0" allowfullscreen></iframe></div>The couple, John Paladino, 73, and Carole Paladino, 72, were the only people inside the home in Newfield, New Jersey when the blast occurred shortly after 6 a.m.(document ID REF-20180707-01; dated July 07, 2018).Flames are seen burning where a New Jersey home exploded early Saturday morning. Two people were killed in the blast, officials said. The cause of the explosion is not currently known. <a href="https://t.co/GA3X6tGIjb">https://t.co/GA3X6tGIjb</a> <a href="https://t.co/7wJt79kqte">pic.twitter.com/7wJt79kqte</a></p>— ABC News (@ABC) <a href="https://twitter.com/ABC/status/1015662071638298625?re

In [142]:
matched['news_cleaned_350'].iloc[2]

'Photo credit: infowars.comNew Jersey couple dies in a home explosion. Caroline Paladino was apparently set to testify to Grand Jury against the Clintons this week..House explosion in New Jersey {Video below}<div style="width:100%;text-align:center;margin:0 auto;"><iframe width="360" height="202" src="https://www.youtube.com/embed/XAOf4QDpnCU" frameborder="0" allowfullscreen></iframe></div>The couple, John Paladino, 73, and Carole Paladino, 72, were the only people inside the home in Newfield, New Jersey when the blast occurred shortly after 6 a.m.<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Flames are seen burning where a New Jersey home exploded early Saturday morning. Two people were killed in the blast, officials said. The cause of the explosion is not currently known. <a href="https://t.co/GA3X6tGIjb">https://t.co/GA3X6tGIjb</a> <a href="https://t.co/7wJt79kqte">pic.twitter.com/7wJt79kqte</a></p>— ABC News (@ABC) <a href="https://twitter.com/ABC/status/1

In [143]:
# Initialize SBERT model
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Helper: clean + sentence split
def split_sentences(text: str):
    text = re.sub(r"\s+", " ", str(text).strip())
    return nltk.sent_tokenize(text)

# Core similarity function (bi-directional sentence matching)
def article_similarity_bidir(a: str, b: str) -> float:
    sents_a, sents_b = split_sentences(a), split_sentences(b)
    if not sents_a or not sents_b:
        return 0.0
    A = model.encode(sents_a, normalize_embeddings=True)
    B = model.encode(sents_b, normalize_embeddings=True)
    S = cosine_similarity(A, B)
    a2b = S.max(axis=1).mean()
    b2a = S.max(axis=0).mean()
    return float((a2b + b2a) / 2)

In [144]:
# Apply to your DataFrame
tqdm.pandas()
matched["semantic_similarity_high_cred_specificity_pr"] = matched.progress_apply(
    lambda row: article_similarity_bidir(row["news_cleaned_350"], row["high_cred_specificity_pr"]),
    axis=1
)

100%|██████████| 78/78 [00:01<00:00, 40.69it/s]
/tmp/ipykernel_1225414/49036244.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched["semantic_similarity_high_cred_specificity_pr"] = matched.progress_apply(


In [145]:
# matched

In [146]:
print(matched["semantic_similarity_high_cred_specificity_pr"].describe())

count    78.000000
mean      0.933033
std       0.090073
min       0.615961
25%       0.892648
50%       0.984885
75%       0.993800
max       0.999505
Name: semantic_similarity_high_cred_specificity_pr, dtype: float64


In [110]:
print(matched["semantic_similarity_sensational"].describe())

count    78.000000
mean      0.817614
std       0.117536
min       0.476346
25%       0.743910
50%       0.828100
75%       0.901687
max       1.000000
Name: semantic_similarity_sensational, dtype: float64
